# Analysis of recent bank activity

- Transactions pulled from 6/1/2022 - 7/16/2022
- Only includes Checking Account Activity

In [ ]:
from abc import ABCMeta, abstractmethod
import csv
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum, IntEnum
import re

import pandas as pd
from pandas import DataFrame
from rich import print
from dataclasses_json import DataClassJsonMixin

## Define Models for working with transactions

In [7]:
DATA_DIRECTORY="/home/scott/Projects/py-finance-parser/data"
RAW_FILEPATH = f"{DATA_DIRECTORY}/raw/stmt.csv"
CLEANSED_FILEPATH = f"{DATA_DIRECTORY}/cleansed/stmt.csv"
PARSED_FILEPATH = f"{DATA_DIRECTORY}/parsed/stmt.json"

class Category(Enum):
    """An Enumeration of categories for transactions to be placed in
    """
    AMAZON = "Amazon"
    AUTO = "Auto"
    DATE_NIGHTS = "Date Nights"
    DELIVERY = "Delivery"
    ELECTRIC = "Electric"
    ENTERTAINMENT = "Entertainment"
    FITNESS = "Gym Fees"
    FRAUD = "Fraud"
    GAS = "Gas"
    GIFTS = "Gifts"
    GROCERIES = "Groceries"
    GRUBHUB = "Grubhub"
    INCOME = "Income"
    INSURANCE = "Insurance"
    INTERNET = "Internet"
    LOANS = "Loans"
    MORTGAGE = "Mortgage"
    PETS = "Pets"
    PHONE = "Phone"
    SAVINGS = "Savings"
    TAKEOUT = "Takeout"
    TAXES = "Taxes"
    WEED = "Weed"
    UNCATEGORIZED = "Uncategorized"

class Column(IntEnum):
    """An enumeration of the column index values for the csv file
    """
    DATE = 0
    DESCRIPTION = 1
    AMOUNT = 2
    RUNNING_BALANCE = 3

class Comparison(Enum):
    GREATER_THAN = ">"
    GREATER_THAN_EQUAL_TO = ">="
    EQUAL_TO = "=="
    NOT_EQUAL_TO = "!="
    LESS_THAN = "<"
    LESS_THAN_EQUAL_TO = "<="

class Month(IntEnum):
    """The Month of the Year starting at 1 for January, up to 12 for December    
    """
    January = 1
    February = 2
    March = 3
    April = 4
    May = 5
    June = 6
    July = 7
    August = 8
    September = 9
    October = 10
    November = 11
    December = 12  


# slots=True tells the data class to auto generate a list of fields for the __slots__ attribute,
# this limits the fields usable for the object to only those present in the list the tradeoff
# being that the system is able to more efficiently allocate these objects an more efficiently 
# access the attributes of the class
@dataclass(slots=True)
class Transaction(DataClassJsonMixin):
    """A dataclass representing a Bank of America Transaction Record
    """
    id: int
    date: datetime
    description: str
    amount: float
    running_balance: float
    category: Category = Category.UNCATEGORIZED
    tags: list[str] = field(default_factory=list)

    @classmethod
    def create_from_csv_row(cls, id, row: list[str]) -> "Transaction":
        month, day, year = row[Column.DATE].split("/")
        return cls(
            id,
            datetime(int(year), int(month), int(day)),
            row[Column.DESCRIPTION],
            float(row[Column.AMOUNT]),
            float(row[Column.RUNNING_BALANCE])
        )
    def __str__(self) -> str:
        return f"Id: {self.id} - Description: {self.description} - Date: {self.date} - Amount: ${self.amount:.2f}"
    def __ne__(self, trx: "Transaction") -> bool:
        return self.id != trx.id

    def __eq__(self, trx: "Transaction") -> bool:
        return self.id == trx.id

    def __gt__(self, trx: "Transaction") -> bool:
        return self.id > trx.id

    def __ge__(self, trx: "Transaction") -> bool:
        return self.id >= trx.id

    def __lt__(self, trx: "Transaction") -> bool:
        return self.id < trx.id

    def __le__(self, trx: "Transaction") -> bool:
        return self.id <= trx.id

@dataclass(slots=True)
class TransactionList(DataClassJsonMixin):
    transactions: list[Transaction]


@dataclass
class Filter(metaclass=ABCMeta):
    name: str
    
    @classmethod
    def __subclasshook__(cls, subclass):
        return (
            (hasattr(subclass, "match") and callable(subclass.match))
            or NotImplemented
        )
    
    @abstractmethod
    def match(self, trx)->bool:
        raise NotImplementedError(self.match)

@dataclass(slots=True)
class DescriptionFilter(DataClassJsonMixin):
    name: str
    category: str
    filter_expression: str    
    tags: list[str]
    
    def __init__(self, name: str, filter_expression: str, category: Category = None, tags: list[str] = []):
        """Initializes a filter that will try to find a match for the regular expression provided and returns a match if true

        Args:
            name (str): the name for the filter
            filter_expression (str): the regular expression to compare against the transaction description
            category (Category): the category associated with this filter
        """
        self.name = name
        self.filter_expression = filter_expression
        self.category = category
        self.tags = tags
    
    def match(self, trx: Transaction)->bool:
        return bool(re.search(self.filter_expression, trx.description))

@dataclass(slots=True)
class ValueFilter:
    name: str
    category: str
    limit: float
    comparison: Comparison
    subfilter: Filter    
    tags: list[str]
    
    def __init__(self, name: str, limit: float, category: Category, comparison: Comparison = Comparison.EQUAL_TO, subfilter: str|Filter = None, tags: list[str] = []):
        """Initialize a ValueFilter, all value filters are applied as follows 'transaction.amount is <comparison> filter.limit', 
            e.g. transaction.amount is GREATER_THAN_OR_EQUAL_TO filter.limit
            
            If a filter expression is also included that will be matched as well, and BOTH must match for the comparison to return True
            NOTE: Comparisons are done against absolute values i.e. +/- signs will be ignored
        Args:
            name (str): The name for this filter
            limit (float): The float value you wish to compare to
            comparison (Comparison): The comparison operation
            category (Category): the category to apply when matching this filter
            subfilter (str|DescriptionFilter, optional): A text phrase to be matched in the transaction's description. Defaults to None.
            order (int, optional): The order the filters should be applied. Defaults to 0.
        """
        if type(subfilter) is str:
            subfilter = DescriptionFilter(f"{name}-subfilter", subfilter, category)
        self.name = name
        self.limit = float(limit)
        self.comparison = comparison
        self.category = category
        self.subfilter = subfilter
        self.tags = tags

    def match(self, trx: Transaction) -> bool:        
        if self.subfilter:
            if not self.subfilter.match(trx):
                return False
        try:
            filter_limit = float(f"{abs(self.limit):2f}")
            trx_amount = float(f"{abs(trx.amount):2f}")
            if self.comparison == Comparison.EQUAL_TO:
                return trx_amount == filter_limit
            elif self.comparison == Comparison.NOT_EQUAL_TO:
                return trx_amount != filter_limit
            elif self.comparison == Comparison.GREATER_THAN:
                return trx_amount < filter_limit
            elif self.comparison == Comparison.GREATER_THAN_EQUAL_TO:
                return trx_amount >= filter_limit
            elif self.comparison == Comparison.LESS_THAN:
                return trx_amount < filter_limit
            elif self.comparison == Comparison.LESS_THAN_EQUAL_TO:
                return trx_amount <= filter_limit
        except Exception as ex:
            print(f"Error occurred with trx: {trx}")
            print(f"filter: {self}")
            print(f"ex: {ex}")
            raise ex
AMAZON_SUBFILTER = DescriptionFilter("Amazon", "AMZN")
VENMO_SUBFILTER = DescriptionFilter("VenmoFilter", "VENMO")
@dataclass(slots=True)
class AmazonFilter(ValueFilter):
    name: str
    category: str
    limit: float
    comparison: Comparison
    subfilter: Filter
    tags: list[str]
    
    def __init__(self, limit: float, category: Category = None, comparison: Comparison = Comparison.EQUAL_TO, tags: list[str] = None):
        self.limit = float(limit)
        self.comparison = comparison
        self.category = category        
        if tags is None:
            tags = []
        self.tags = tags
        self.subfilter = AMAZON_SUBFILTER


@dataclass(slots=True)
class DateRangeFilter:
    from_date: datetime
    to_date: datetime
    subfilter: Filter
    _tags: list[str] = field(default_factory=list)

    @property
    def tags(self)->list[str]:
        return list(set(self.tags + self.subfilter.tags))
    
    @property
    def category(self)->Category:
        return self.subfilter.category
    
    def match(self, trx: Transaction):
        if self.from_date <= trx.date <= self.to_date:
            return self.subfilter.match(trx)
        return False

## Load the data set

In [8]:
with open(RAW_FILEPATH, 'r+') as f:
    raw_dataset = [row for row in csv.reader(f, dialect='excel')]
dataset = raw_dataset[8:]

## Clean up data
- Remove header rows
- Remove commas
- Convert empty values for amount and balance to 0.0
- Save the clensed data

In [9]:
for row in dataset:
    if row[Column.AMOUNT] == "":
        row[Column.AMOUNT] = "0.0"
    if row[Column.RUNNING_BALANCE] == "":
        row[Column.RUNNING_BALANCE] = "0.0"
    row[Column.RUNNING_BALANCE] = row[Column.RUNNING_BALANCE].replace(",","")
    row[Column.AMOUNT] = row[Column.AMOUNT].replace(",","")

with open(CLEANSED_FILEPATH, "w+", newline="") as f:
    writer = csv.writer(f, dialect="excel")
    writer.writerows(dataset)    

## Parse Data
- Parse data into Transaction class
- Save results to json file

In [10]:
parsed_transactions = TransactionList([Transaction.create_from_csv_row(i + 1, row) for i, row in enumerate(dataset)])

with open(PARSED_FILEPATH, "w+") as f:
    f.write(parsed_transactions.to_json())

## Create and Apply Filters
- Create list of filters

In [11]:

filters: list[DescriptionFilter] = [
    DateRangeFilter(datetime(2022, 6, 5), datetime(2022, 6, 7), AmazonFilter(27.87, Category.FRAUD)),
    DateRangeFilter(datetime(2022, 6, 10), datetime(2022, 7, 12), AmazonFilter(8.50, Category.FRAUD)),
    DateRangeFilter(datetime(2022, 6, 20), datetime(2022, 6, 22), AmazonFilter(9.56, Category.FRAUD)),    
    ValueFilter("Chris", 120.0, Category.WEED, subfilter=VENMO_SUBFILTER),
    ValueFilter("Aza", 15.0, Category.GIFTS, subfilter=VENMO_SUBFILTER),
    DescriptionFilter("GrubHub", "GRUBHUB", Category.DELIVERY),
    DescriptionFilter("PayCheck", "DIRECT DEP", Category.INCOME),
    DescriptionFilter("Snacks", "CHESHIRE GAS", Category.GROCERIES),
    DescriptionFilter("Steam", "STEAMGAMES", Category.ENTERTAINMENT),
    DescriptionFilter("McCue-2", "MCCUE", Category.MORTGAGE),
    DescriptionFilter("AllState", "ALLSTATE", Category.INSURANCE),
    DescriptionFilter("Gym", "TENNIS", Category.FITNESS),
    DescriptionFilter("Gym-2", "EDGE\sFITNESS", Category.FITNESS),
    DescriptionFilter("SavingsDeposit", "transfer\sto\sSAV", Category.SAVINGS),
    DescriptionFilter("Mortgage", "McCue", Category.MORTGAGE),
    DescriptionFilter("PSN", "PLAYSTATION", Category.ENTERTAINMENT),
    DescriptionFilter("Dunkin", "DUNKIN", Category.TAKEOUT),
    DescriptionFilter("Snacks", "SAM'S\sFOOD", Category.TAKEOUT),
    DescriptionFilter("KeepTheChange", "KEEP\sTHE\sCHANGE", Category.SAVINGS),
    DescriptionFilter("SavingsWithdraw", "transfer\sfrom\sSAV", Category.SAVINGS),
    DescriptionFilter("Comcast", "COMCAST", Category.INTERNET),
    DescriptionFilter("Att", "ATT\sDES", Category.PHONE),
    DescriptionFilter("Peapod", "PEAPOD", Category.GROCERIES),
    DescriptionFilter("Patreon", "PATREON\sMEMBER", Category.ENTERTAINMENT),
    DescriptionFilter("Juli'sBills", "MAGRATH", Category.LOANS),
    DescriptionFilter("Juli'sBills-2", "Magrath", Category.LOANS),
    DescriptionFilter("PetSupplies", "PETCO", Category.PETS),
    DescriptionFilter("HouseWork", "SM\sMECHANICAL\sSERVICES", Category.GAS),
    DescriptionFilter("Aresco", "ARESCO", Category.GROCERIES),
    DescriptionFilter("Sunoco", "SUNOCO", Category.GROCERIES),
    DescriptionFilter("Affirm", "AFFIRM", Category.LOANS),
    DescriptionFilter("LifeInsurance", "NEW\sYORK\sLIFE\sDES", Category.INSURANCE),
    DescriptionFilter("Kindle", "KINDLE", Category.ENTERTAINMENT),
    DescriptionFilter("YouTube", "YOUTUBE", Category.ENTERTAINMENT),
    DescriptionFilter("Fuel", "CITGO", Category.AUTO),
    DescriptionFilter("AMEXCreditCard", "AMERICA\sCREDIT\sCARD", Category.LOANS),
    DescriptionFilter("MCCreditCard", "PAYPAL\sEXTRAS\sMASTERCARD", Category.LOANS),
    DescriptionFilter("Bilaton", "STRYVEFOODS", Category.GROCERIES),
    DescriptionFilter("Theater", "THOMASTON\sOPERA\sHOUSE", Category.ENTERTAINMENT),
    DescriptionFilter("Electric", "CL&P", Category.ELECTRIC),
    DescriptionFilter("PriceChopper", "PRICE\sCHOPPER", Category.GROCERIES),
    DescriptionFilter("IRS", "IRS\sDES", Category.TAXES),
    DescriptionFilter("Vivint", "VIVINT", Category.INSURANCE),
    DescriptionFilter("Dropbox", "DROPBOX", Category.INTERNET),
    DescriptionFilter("VPN", "MOZILLACORP", Category.INTERNET),
    DescriptionFilter("Fairview", "FAIRVIEW", Category.GIFTS),
    DescriptionFilter("UpgradeLoad", "UPGRADE", Category.LOANS),
    DescriptionFilter("Eversource", "EVERSOURCE", Category.GAS),
    DescriptionFilter("CarLoan", "CAPITAL\sONE\sAUTO", Category.LOANS),
    DescriptionFilter("Chris", "TO\sCHRIS", Category.WEED),
    DescriptionFilter("Aza", "TO\sAZA", Category.ENTERTAINMENT),
    DescriptionFilter("Stryve-2", "STRYVE", Category.GROCERIES),
    DescriptionFilter("GasStation", "FUEL\sPLUS", Category.AUTO),
    DescriptionFilter("NetFlix", "Netflix", Category.ENTERTAINMENT), 
    AmazonFilter(10.62, Category.ENTERTAINMENT), 
    AmazonFilter(11.29, Category.GROCERIES),
    AmazonFilter(53.15, Category.ENTERTAINMENT),
    AmazonFilter(16.84, Category.GROCERIES),
    AmazonFilter(14.13, Category.WEED),
    AmazonFilter(26.12, Category.GROCERIES),
    AmazonFilter(57.40, Category.GROCERIES),
    AmazonFilter(19.30, Category.GROCERIES),
    AmazonFilter(180.78, Category.ENTERTAINMENT),
    AmazonFilter(17.01, Category.ENTERTAINMENT),
    
]

- Automatically categorize Income as anything > $0.00 
- Run uncategorized transactions against filters, matches get category set by filter

In [12]:

for trx in parsed_transactions.transactions:
    if trx.amount >= 0.01:
        trx.category = Category.INCOME

for filter in filters:
    uncategorized = [trx for trx in parsed_transactions.transactions if trx.category == Category.UNCATEGORIZED]
    if not uncategorized:
        break
    for trx in uncategorized:
        if filter.match(trx):
            trx.category = filter.category

categorized = [trx for trx in parsed_transactions.transactions if trx.category != Category.UNCATEGORIZED]
uncategorized = [trx for trx in parsed_transactions.transactions if trx.category == Category.UNCATEGORIZED]
print(f"There are {len(categorized)} categorized transactions")
print(f"There are {len(uncategorized)} uncategorized transactions")

if uncategorized:
    length = 20
    if len(uncategorized) < 20:        
        length = len(uncategorized)
    print(f"Top {length} Uncategorized Transactions")
    for trx in uncategorized[:length]:
        print(trx)
amazon_items = [trx for trx in parsed_transactions.transactions if trx.category == Category.AMAZON]
if amazon_items:
    print("----Amazon Items----")
for item in [trx for trx in amazon_items]:
    print(trx)

There are 187 categorized transactions
There are 0 uncategorized transactions


## Save the results

In [13]:
with open(PARSED_FILEPATH, "w") as f:
    f.write(parsed_transactions.to_json())

## Analysis of Categorized Transactions

In [14]:
CATEGORY_HEADER_TEXT = "Category"
ZERO_TOTAL_TEXT = "$0.00"
with open(PARSED_FILEPATH, "r+") as f:
    trx_list = TransactionList.from_json(f.read())

totals_by_category_by_month: dict[Month, dict[Category, float]] = {}
months = set([Month(trx.date.month) for trx in trx_list.transactions])
for month in months:
    totals_by_category = {}    
    for trx in trx_list.transactions:    
        if trx.date.month == month:
            if trx.category not in totals_by_category.keys():
                totals_by_category[trx.category] = trx.amount
            else:
                totals_by_category[trx.category] += trx.amount
    totals_by_category_by_month[month] = totals_by_category
    
totals_by_category: dict[Category, dict[Month, float]] = {}
for month, totals_for_month in totals_by_category_by_month.items():    
    for category, total in totals_for_month.items():
        if category not in totals_by_category.keys():
            totals_by_category[category] = {Month(month): total}
        else:
            totals_by_category[category][Month(month)] = total

header_row: list[str] = [
    CATEGORY_HEADER_TEXT
]
for month in months:
    header_row.append(month.name)

data: list[list[str]] = []
for category, month_totals in totals_by_category.items():
    row = [category.value]
    for month in months:
        if month in month_totals.keys():            
            total = month_totals[month]
            row.append(f"${total:.2f}")
        else:
            row.append(ZERO_TOTAL_TEXT)
    data.append(row)

DataFrame(data, columns=header_row)

,Category,June,July
0,Income,$9557.82,$3377.60
1,Delivery,$-489.47,$-511.97
2,Entertainment,$-424.69,$-150.97
3,Gym Fees,$-50.99,$-71.21
4,Takeout,$-117.39,$-105.46
5,Savings,$-1398.33,$-120.58
6,Groceries,$-839.72,$-407.74
7,Mortgage,$-1275.00,$-1275.00
8,Fraud,$-45.93,$-8.50
9,Phone,$-440.88,$0.00
